In [ ]:
import os
import glob
from osgeo import gdal

# Définir les chemins des fichiers et des dossiers
band_path = "data/images"
band_files = os.ptha.join(band_path, "SENTINEL2B*.tif")
out_path = "Projet_Teledetection/results/img_pretraitees"
mask_file = os.path.join("masque_foret.tif") 

# Paramètres de sortie
output_file = os.path.join(out_path, "Serie_temp_S2_allbands.tif")
projection = "EPSG:2154"
pixel_size = 10  # Résolution spatiale en mètres
nodata_value = 0

# Fonction pour reprojeter et rééchantillonner une bande
def reproject_and_resample(input_file, output_file, pixel_size, projection):
    gdal.Warp(
        output_file,
        input_file,
        xRes=pixel_size,
        yRes=pixel_size,
        dstSRS=projection,
        dstNodata=nodata_value
    )

# Étape 1 : Reprojeter et rééchantillonner toutes les bandes
reprojected_bands = []
for i, band_file in enumerate(band_files):
    reprojected_file = os.path.join(out_path, f"band_{i+1}_reprojected.tif")
    reproject_and_resample(band_files, reprojected_file, pixel_size, projection)
    reprojected_bands.append(reprojected_file)

# Étape 2 : Appliquer le masque forêt à chaque bande
masked_bands = []
for i, band_files in enumerate(reprojected_bands):
    masked_file = os.path.join(out_path, f"band_{i+1}_masked.tif")
    gdal_calc_command = (
        f"gdal_calc.py -A {band_file} -B {mask_file} --outfile={masked_file} "
        f"--calc=\"A*B\" --NoDataValue={nodata_value} --quiet"
    )
    os.system(gdal_calc_command)
    masked_bands.append(masked_file)

# Étape 3 : Combiner toutes les bandes dans un seul fichier
merge_command = (
    f"gdal_merge.py -separate -o {output_file} " + " ".join(masked_bands) + f" -a_nodata {nodata_value}"
)
os.system(merge_command)

# Étape 4 : Vérification et conversion optionnelle en uint8
output_uint8_file = output_file.replace(".tif", "_uint8.tif")
if os.path.getsize(output_file) > 10 * 1024 * 1024 * 1024:  # 10 Go
    dynamic_convert_command = (
        f"otbcli_DynamicConvert -in {output_file} -out {output_uint8_file} uint8"
    )
    os.system(dynamic_convert_command)
    print(f"Fichier converti en uint8 : {output_uint8_file}")
else:
    print(f"Fichier final généré : {output_file}")


KeyError: 'Code'